In [1]:
import pandas as pd
import random
from collections import Counter
import numpy as np
# !pip3 install -q sklearn
from sklearn.linear_model import LinearRegression
# import tensorflow as tf
# import tensorflow.keras as keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, LSTM, Dropout

In [2]:
xcelLocation = "../xcels/"
# !export xcelLocation="../xcels/"
csvFileName = "MarketWatchPlus-1398_5_7"
csvFileName = "1382-2-2"
RECORD_THRESHOLD = 3000
TESTCASE_NUMBER = 20
alreadyPrep = False

In [3]:
!ls $xcelLocation | grep ".csv" > csvFiles
tmp = !cat csvFiles
csvFiles = [name[:-4] for name in tmp]

In [4]:
def readCsv(xcelLocation, csvFileName):
    df=pd.read_csv(xcelLocation + csvFileName + '.csv', sep=',',header=[0])
    return df

def addDiff(df): 
    diffTable = df.loc[:,["نام","کمترین","بیشترین"]]
    sumdif,count = (0,0)
    difs = []
    for row in range(len(diffTable)):
        minim = diffTable.loc[row][1]
        maxim = diffTable.loc[row][2]
        if(minim != 0):
            count += 1
            sumdif += (maxim - minim)/minim
            difs.append((maxim - minim)/minim * 100)
        else:
            difs.append(0)
    diffTable.loc[:,"diffs"] = difs
    df.loc[:,"diffs"] = difs
    return df, diffTable, sumdif, count

In [5]:
def prepare():
    global alreadyPrep
    global df
    global minMaxDF
    global testNames
    
    if alreadyPrep:
        return
    alreadyPrep = True
    
    df = readCsv(xcelLocation=xcelLocation, csvFileName="master")
    minMaxDF = df.loc[:,["نام","کمترین","بیشترین", "قیمت پایانی - مقدار", "year", "month", "day"]]
    minMaxDF.columns = ["name", "min", "max", "close", "year", "month", "day"]
    
    allNames = minMaxDF.name.tolist()
    names = sorted(list(set(minMaxDF.name))[1:])
    print(len(names))
    counts = Counter(allNames)
    testNames = []
    tmpNames = []
    for name in names:
        if counts[name] > RECORD_THRESHOLD:
            tmpNames.append(name)
    for i in range(TESTCASE_NUMBER):
        ran = random.randint(0, len(tmpNames))
        testNames.append(tmpNames[ran])
        tmpNames.remove(tmpNames[ran])
    print(len(testNames))

In [125]:
def runSimulator(testDF, minMaxPredictFunction):
    ans = testDF.iloc[len(testDF) - 1]
    testDF = testDF.drop(len(testDF) - 1)
    predictMin, predictMax = minMaxPredictFunction(testDF)

    result = 0
    if predictMin >= ans["min"] and predictMin <= ans["max"]:
        if predictMax <= ans["max"] and predictMax >= ans["min"]:
            result += 1
        else:
            result += 2
    return result

In [126]:
def calcRate(minMaxPredictFunction):
    prepare()
    SS = 0
    SF = 0
    FF = 0
    
    for name in testNames:
        testDF = minMaxDF.loc[minMaxDF["name"] == name].reset_index(drop=True)
        for i in range(200):
            result = runSimulator(testDF, minMaxPredictFunction)
            if result % 2 == 1:
                SS += 1
            if (result/2) % 2 == 1:
                SF += 1
            if result == 0:
                FF += 1
            testDF = testDF.drop([len(testDF) - 1, len(testDF) - 2, len(testDF) - 3, len(testDF) - 4])
    print(SS, " ", SF, " ", FF)
    print("prec ", SS/(SS + SF))
    print("rec ", (SS + SF)/(SS + FF + SF))
            

In [127]:
def rator1(inputDF):
    size = len(inputDF)
    return inputDF.iloc[size - 1]["min"]*102/100 , inputDF.iloc[size - 1]["max"]

In [128]:
def rator2(inputDF):
    regressor = Sequential()
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))

    regressor.add(Dense(units = 1))

    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

    regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [130]:
calcRate(rator1)

3128
20
888   787   2325
prec  0.5301492537313433
rec  0.41875


In [ ]:
def addDiff(df): 
    diffTable = df.loc[:,["نام","کمترین","بیشترین"]]
    sumdif,count = (0,0)
    difs = []
    for row in range(len(diffTable)):
        minim = diffTable.loc[row][1]
        maxim = diffTable.loc[row][2]
        if(minim != 0):
            count += 1
            sumdif += (maxim - minim)/minim
            difs.append((maxim - minim)/minim * 100)
        else:
            difs.append(0)
    diffTable.loc[:,"diffs"] = difs
    df.loc[:,"diffs"] = difs
    return df, diffTable, sumdif, count

In [29]:
def alarm1():
    prepare()
    difs = []
    minMax98 = minMaxDF.loc[minMaxDF["year"] == 1398].reset_index(drop=True)
    for row in range(len(minMax98)):
        minim = minMax98.loc[row][1]
        maxim = minMax98.loc[row][2]
        if(minim != 0):
            difs.append((maxim - minim)/minim * 100)
        else:
            difs.append(0)
    minMax98.loc[:,"diffs"] = difs
    heat5DF = minMax98.loc[minMax98["diffs"] > 4.5].reset_index(drop=True)
    return heat5DF

In [30]:
heat5DF = alarm1()

In [47]:
less5 = heat5DF.loc[heat5DF["diffs"] <= 5].reset_index(drop=True)
more4 = less5.loc[less5["diffs"] > 4.8].reset_index(drop=True)
more4.loc[more4["month"] == 5].reset_index(drop=True)

,name,min,max,close,year,month,day,diffs
0,اعتباري ملل,2262.0,2375.0,2346.0,1398,5,15,4.995579
1,پلي اكريل ايران,6860.0,7195.0,7007.0,1398,5,15,4.883382
2,ايران‌ ترانسفو,6905.0,7245.0,7067.0,1398,5,15,4.923968
3,ملي‌ صنايع‌ مس‌ ايران‌,4050.0,4249.0,4161.0,1398,5,15,4.913580
4,سيمان فارس و خوزستان,3470.0,3640.0,3545.0,1398,5,15,4.899135
5,صنعتي‌ بهشهر,7708.0,8086.0,8018.0,1398,5,15,4.903996
6,سرمايه‌گذاري صنايع پتروشيمي‌,4000.0,4196.0,4168.0,1398,5,15,4.900000
7,داروسازي توليد دارو,3882.0,4076.0,4010.0,1398,5,15,4.997424
8,البرزدارو,7025.0,7364.0,7279.0,1398,5,15,4.825623
9,لوازم‌ خانگي‌ پارس‌,31998.0,33535.0,33207.0,1398,5,15,4.803425
